In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing(as_frame=True, return_X_y=True)
df, target = data[0], data[1] 
df = StandardScaler().fit_transform(df)

cv = KFold(n_splits=4, shuffle=True, random_state=6)

In [2]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

ridge1 = Ridge(alpha=1)
predictions1 = cross_val_predict(ridge1, df, target, cv=cv)

ridge10 = Ridge(alpha=10)
predictions10 = cross_val_predict(ridge10, df, target, cv=cv)

mse1 = mean_squared_error(target, predictions1, squared=False)
mse10 = mean_squared_error(target, predictions10, squared=False)

print("RMSE для Ridge с alpha=1:", mse1)
print("RMSE для Ridge с alpha=10:", mse10)

RMSE для Ridge с alpha=1: 0.7283772378148317
RMSE для Ridge с alpha=10: 0.7283219021033459


In [8]:
import numpy as np

def sign_test(x, y):
    n = int(x)
    signs = np.sign(x - y)
    n_plus = np.sum(signs > 0)
    n_minus = np.sum(signs < 0)
    print('hi')
    
    p_value = 2 * min(min(n_plus, n_minus), 0.5*n)
    return p_value

p_value = sign_test(mse1, mse10)
print("p-value для сравнения медиан:", p_value)

if p_value < 0.05:
    print("Различие медиан статистически значимо.")
else:
    print("Нет статистически значимого различия медиан.")


hi
p-value для сравнения медиан: 0
Различие медиан статистически значимо.


Некоторые непараметрические статистические критерии делают переход к рангам.

Посчитайте ранги для элементов следующей выборки. Ранги равных по значениям элементов считайте так же, как мы это делали на уроке.

[7, 1, 5, 1, 3, 2, 5]

In [9]:
import scipy.stats
scipy.stats.rankdata([7, 1, 5, 1, 3, 2, 5], method='average')

array([7. , 1.5, 5.5, 1.5, 4. , 3. , 5.5])

В критерии Манна-Уитни-Уилкоксона распределение статистики табличное. Статистикой выступает сумма рангов первой выборки в объединённой выборке.

Представим, что у нас две выборки: первая размера 2, а вторая размера 3. Все значения в выборках различаются.

Постройте соответствующее этому случаю табличное распределение и ответьте на вопрос, с какой вероятностью мы могли встретить значения статистики 7 и более среди всех возможных значений статистики при верности нулевой гипотезы о том, что распределения не различаются.

Это задание удобнее делать на листочке без написания кода.

In [ ]:
#решение: 0.4

В прошлом уроке мы сравнивали средние доходы (MedInc) между районами, где дома моложе 30 лет (включительно) и старше 30 лет (HouseAge). Однако тогда мы заметили, что случайная величина распределена не нормально.

Давайте воспользуемся непараметрическими статистическими критериями, которые не делают предположений на вид распределения.

Примените критерий Манна-Уитни-Уилкоксона к выборке и сравните, различаются ли распределения между выборками. Какой вывод вы можете сделать? Альтернатива двусторонняя.

In [10]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True, return_X_y=True)
df, target = data[0], data[1] 

In [11]:
import pandas as pd

df_wt = pd.concat([df, target], axis=1)
income_district_1 = df_wt[df_wt["HouseAge"] <= 30]
income_district_2 = df_wt[df_wt["HouseAge"] > 30]

In [16]:
import numpy as np
from scipy.stats import mannwhitneyu

statistic, p_value = mannwhitneyu(income_district_1, income_district_2, alternative='two-sided')


print("Статистика критерия:", statistic)
print("p-value:", p_value)

Статистика критерия: [59254034.5        0.  64370004.5 60117703.  68340137.  52169558.
 52772225.5 58601548.5 49651963.5]
p-value: [5.34519869e-050 0.00000000e+000 6.63212500e-159 5.15048228e-064
 2.20078456e-286 8.22820824e-002 7.45171789e-001 1.38430158e-040
 2.18979809e-014]


Представим, что мы хотели бы зафиксировать ошибку первого рода при применении статистического критерия на уровне 1%. У нас есть 12 экспериментальных моделей, среди которых мы хотим найти те, которые будут отличаться от текущей в лучшую сторону.

Какая ошибка первого рода «хотя бы одна из моделей случайно покажется лучше контрольной» у нас будет на самом деле? Результаты всех моделей независимы друг от друга.

In [ ]:
#решение:11.4%